# Wilcoxon SIGNED test - ACCURACY
By: Sam<br>
Updated at: 16/10/2022<br>
Compare performance of discretizers <br>
Matched pairs settings
Sample: 270 ML models after discretization (as at 16.10.2022)
Purpose: pair-wise comparison metrics of the models using different discretization method

===

Input data: instrinsic properties and model performance metrics
!!! **NB: Please update the data for metrics and export to csv before running this script!

In [1]:
# Import library
import pandas as pd
import numpy as np
from scipy import stats
import math
import random

In [2]:
# Import evaluation data (updated at 16/10/2022)
data = pd.read_csv("all_evaluation_161022.csv")

In [3]:
data.head()

,dataset,disc,param,inconsistency,models,accuracy,con_features,time_disc,time_train,bias,variance
0,iris,EWD,4,0.06666667,ID3,0.84,4,0.016412,0.008698225,0.158,0.055
1,iris,EWD,7,0.02,ID3,0.79,4,0.015692,0.010634899,0.158,0.054
2,iris,EWD,10,0.006666667,ID3,0.95,4,0.016380,0.010643005,0.053,0.014
3,iris,EFD,4,0.04,ID3,0.84,4,0.016688,0.009439945,0.158,0.049
4,iris,EFD,7,0.04,ID3,0.95,4,0.023941,0.010675907,0.053,0.07


In [4]:
data['disc'].unique() # get list of discretizers

array(['EWD', 'EFD', 'FFD', 'ChiMerge', 'DT'], dtype=object)

In [5]:
data.columns

Index(['dataset', 'disc', 'param', 'inconsistency', 'models', 'accuracy',
       'con_features', 'time_disc', 'time_train', 'bias', 'variance'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   dataset        270 non-null    object 
 1   disc           270 non-null    object 
 2   param          270 non-null    int64  
 3   inconsistency  270 non-null    object 
 4   models         270 non-null    object 
 5   accuracy       270 non-null    object 
 6   con_features   270 non-null    int64  
 7   time_disc      270 non-null    float64
 8   time_train     269 non-null    object 
 9   bias           234 non-null    object 
 10  variance       234 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 23.3+ KB


In [7]:
data['models'].unique()

array(['ID3', 'CNB', 'Knn-VDM'], dtype=object)

# Wilcoxon_Complete pipeline

## 1.  Wilcoxon signed t test, accuracy
Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html
Implement 4 replications: (DONE)
- Regardless algorithms
- Filter for each algorithm: CNB, ID3, Knn

## 1. Accuracy, no filter in algorithm

In [8]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain accuracy for each discretization, convert into numeric, string values will be return as NaN
ewd_acc = pd.to_numeric(data[data['disc']=="EWD"]['accuracy'],errors='coerce').tolist()
efd_acc = pd.to_numeric(data[data['disc']=="EFD"]['accuracy'],errors='coerce').tolist()
ffd_acc = pd.to_numeric(data[data['disc']=="FFD"]['accuracy'],errors='coerce').tolist()
cm_acc = pd.to_numeric(data[data['disc']=="ChiMerge"]['accuracy'],errors='coerce').tolist()
dt_acc = pd.to_numeric(data[data['disc']=="DT"]['accuracy'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
# print(len(ewd_acc))
# print(len(efd_acc))
# print(len(ffd_acc))
# print(len(cm_acc))
# print(len(dt_acc))

# Step 2: filter numeric values
raw_list = [ewd_acc, efd_acc, ffd_acc, cm_acc, dt_acc]
num_list = [] # filter numeric values only
for metric in raw_list:
#     metric_new = []
#     for x in metric:
#         if math.isnan(x) == False:
#             metric_new.append(x)
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    #print(metric_new)
    #print(len(metric_new))
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

In [9]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of accuracy
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                #print(disc[i][m])
                #print(disc[j][m])
#                 print('diff = ', diff)
#                 print('-------------')
                diff_list.append(diff)
                # print(diff_list)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)
# print(test_stat)
# print(p_value)

In [10]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [11]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [12]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,297.0,0.571536
1,ewd vs ffd,293.5,0.381443
2,ewd vs cm,329.5,0.551990
3,ewd vs dt,293.0,0.377250
5,efd vs ffd,351.5,0.782824
6,efd vs cm,347.0,0.945856
7,efd vs dt,273.5,0.159291
10,ffd vs cm,357.5,0.850432
11,ffd vs dt,157.0,0.045349
15,cm vs dt,235.0,0.078696


## 1.2 Accuracy, only CNB

In [13]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain accuracy for each discretization, convert into numeric, string values will be return as NaN
# Filter for CNB
# Obtain accuracy for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_cnb = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="CNB")]['accuracy'],errors='coerce').tolist()
efd_acc_cnb = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="CNB")]['accuracy'],errors='coerce').tolist()
ffd_acc_cnb = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="CNB")]['accuracy'],errors='coerce').tolist()
cm_acc_cnb = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="CNB")]['accuracy'],errors='coerce').tolist()
dt_acc_cnb = pd.to_numeric(data[(data['disc']=="DT") & (data['models']=="CNB")]['accuracy'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc_cnb))
print(len(efd_acc_cnb))
print(len(ffd_acc_cnb))
print(len(cm_acc_cnb))
print(len(dt_acc_cnb))

# Step 2: filter numeric values
raw_list = [ewd_acc_cnb,efd_acc_cnb, ffd_acc_cnb, cm_acc_cnb, dt_acc_cnb]
num_list = [] # filter numeric values only
for metric in raw_list:
#     metric_new = []
#     for x in metric:
#         if math.isnan(x) == False:
#             metric_new.append(x)
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    #print(metric_new)
    #print(len(metric_new))
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

15
15
20
20
20


In [14]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of accuracy
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                #print(disc[i][m])
                #print(disc[j][m])
#                 print('diff = ', diff)
#                 print('-------------')
                diff_list.append(diff)
                # print(diff_list)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)
# print(test_stat)
# print(p_value)

/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [15]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [16]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [17]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,38.5,0.968700
1,ewd vs ffd,52.0,1.000000
2,ewd vs cm,25.0,0.151205
3,ewd vs dt,36.5,0.325806
6,efd vs cm,22.0,0.100317
7,efd vs dt,23.5,0.123830
10,ffd vs cm,38.0,0.390991
11,ffd vs dt,26.5,0.326613
15,cm vs dt,21.5,0.305838
16,dt vs ewd,36.5,0.357544


## 1.2 Accuracy, only ID3

In [18]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain accuracy for each discretization, convert into numeric, string values will be return as NaN
# Filter for ID3
# Obtain accuracy for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_ID3 = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="ID3")]['accuracy'],errors='coerce').tolist()
efd_acc_ID3 = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="ID3")]['accuracy'],errors='coerce').tolist()
ffd_acc_ID3 = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="ID3")]['accuracy'],errors='coerce').tolist()
cm_acc_ID3 = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="ID3")]['accuracy'],errors='coerce').tolist()
dt_acc_ID3 = pd.to_numeric(data[(data['disc']=="DT") & (data['models']=="ID3")]['accuracy'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc_ID3))
print(len(efd_acc_ID3))
print(len(ffd_acc_ID3))
print(len(cm_acc_ID3))
print(len(dt_acc_ID3))

# Step 2: filter numeric values
raw_list = [ewd_acc_ID3,efd_acc_ID3, ffd_acc_ID3, cm_acc_ID3, dt_acc_ID3]
num_list = [] # filter numeric values only
for metric in raw_list:
#     metric_new = []
#     for x in metric:
#         if math.isnan(x) == False:
#             metric_new.append(x)
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    #print(metric_new)
    #print(len(metric_new))
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

15
15
20
20
20


In [19]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of accuracy
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                #print(disc[i][m])
                #print(disc[j][m])
#                 print('diff = ', diff)
#                 print('-------------')
                diff_list.append(diff)
                # print(diff_list)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)
# print(test_stat)
# print(p_value)

/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [20]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [21]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [22]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,16.0,0.778106
1,ewd vs ffd,41.5,0.302795
2,ewd vs cm,29.5,0.263348
3,ewd vs dt,18.5,0.015076
5,efd vs ffd,39.5,0.252380
6,efd vs cm,27.5,0.207996
7,efd vs dt,22.0,0.030151
8,ffd vs ewd,41.5,0.330261
9,ffd vs efd,39.5,0.276855
10,ffd vs cm,22.0,0.952260


## 1.2 Accuracy, only KNN-VDM


In [23]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain accuracy for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain accuracy for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="Knn-VDM")]['accuracy'],errors='coerce').tolist()
efd_acc_knn = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="Knn-VDM")]['accuracy'],errors='coerce').tolist()
ffd_acc_knn = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="Knn-VDM")]['accuracy'],errors='coerce').tolist()
cm_acc_knn = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="Knn-VDM")]['accuracy'],errors='coerce').tolist()
dt_acc_knn = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="Knn-VDM")]['accuracy'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn))
print(len(efd_acc_knn))
print(len(ffd_acc_knn))
print(len(cm_acc_knn))
print(len(dt_acc_knn))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn,efd_acc_knn, ffd_acc_knn, cm_acc_knn, dt_acc_knn]
num_list = [] # filter numeric values only
for metric in raw_list:
#     metric_new = []
#     for x in metric:
#         if math.isnan(x) == False:
#             metric_new.append(x)
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    #print(metric_new)
    #print(len(metric_new))
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

15
15
20
20
20


In [24]:
print(num_list)

[[0.87, 0.97, 0.95, 0.83, 0.89, 0.91, 0.79, 0.77, 0.68], [0.92, 0.95, 0.95, 0.89, 0.87, 0.92, 0.92, 0.92, 0.77, 0.77, 0.77, 0.68, 0.7, 0.71], [0.97, 0.95, 0.84, 0.84, 0.77, 0.8, 0.79, 0.81, 0.71, 0.71, 0.69, 0.68], [0.95, 0.95, 0.31, 0.23, 0.22, 0.22, 0.91, 0.9, 0.91, 0.88, 0.71, 0.8, 0.77, 0.79], [0.95, 0.95, 0.31, 0.23, 0.22, 0.22, 0.91, 0.9, 0.91, 0.88, 0.71, 0.8, 0.77, 0.79]]


In [25]:
print(k)

9


In [26]:
print(test_list)

[[0.87, 0.97, 0.95, 0.83, 0.89, 0.91, 0.79, 0.77, 0.68], [0.68, 0.77, 0.95, 0.87, 0.95, 0.92, 0.71, 0.92, 0.89], [0.68, 0.69, 0.84, 0.77, 0.95, 0.79, 0.71, 0.71, 0.81], [0.8, 0.71, 0.31, 0.22, 0.95, 0.22, 0.79, 0.95, 0.23], [0.8, 0.71, 0.31, 0.22, 0.95, 0.22, 0.79, 0.95, 0.23]]


In [27]:
print(len(test_list))

5


In [28]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of accuracy
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

In [29]:
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
#                 print(disc[i][m])
#                 print(disc[j][m])
#                 print('diff = ', diff)
#                 print('-------------')
                diff_list.append(diff)
                # print(diff_list)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)
# print(test_stat)
# print(p_value)


/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [30]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [31]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [32]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,17.0,0.888638
1,ewd vs ffd,8.5,0.128906
2,ewd vs cm,4.0,0.04995
5,efd vs ffd,0.0,0.027708
6,efd vs cm,8.0,0.161429
8,ffd vs ewd,8.5,0.097656
10,ffd vs cm,10.0,0.262618
15,cm vs dt,N/A,N/A
